In [30]:
!pip install pytesseract pdf2image pillow pdfplumber pandas


  Using cached cffi-2.0.0-cp313-cp313-win_amd64.whl.metadata (2.6 kB)
  Using cached pycparser-2.23-py3-none-any.whl.metadata (993 bytes)
   ---------------------------------------- 0.0/6.6 MB ? eta -:--:--
   ------ --------------------------------- 1.0/6.6 MB 5.2 MB/s eta 0:00:02
   ----------- ---------------------------- 1.8/6.6 MB 4.6 MB/s eta 0:00:02
   --------------- ------------------------ 2.6/6.6 MB 4.3 MB/s eta 0:00:01
   -------------------- ------------------- 3.4/6.6 MB 4.2 MB/s eta 0:00:01
   --------------------------- ------------ 4.5/6.6 MB 4.1 MB/s eta 0:00:01
   ------------------------------- -------- 5.2/6.6 MB 4.1 MB/s eta 0:00:01
   ------------------------------------ --- 6.0/6.6 MB 4.1 MB/s eta 0:00:01
   ---------------------------------------- 6.6/6.6 MB 4.0 MB/s  0:00:01
   ---------------------------------------- 0.0/3.5 MB ? eta -:--:--
   -------- ------------------------------- 0.8/3.5 MB 4.0 MB/s eta 0:00:01
   ----------------- ----------------------


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pdfplumber
import re

def extract_medical_parameters(pdf_path):
    # Dictionary to store results
    extracted_data = {
        "Glucose": None,
        "Insulin": "Not Found",
        "Creatinine": None,
        "Urea": None,
        "BUN": None,
        "Sodium": None,
        "Potassium": None,
        "Hemoglobin": None,
        "WBC": None,
        "Lactate": "Not Found",
        "pH": None,
        "Age": None,
        "Gender": None,
        "Cholesterol": None
    }

    # Open the PDF
    with pdfplumber.open(pdf_path) as pdf:
        full_text = ""
        # Combine text from all pages to ensure we catch everything
        for page in pdf.pages:
            full_text += page.extract_text() + "\n"

    # --- REGEX PATTERNS ---
    # These patterns are tuned to the specific layout of your provided report
    
    # 1. Demographics (Sex/Age : Male / 41 Y)
    age_gender_match = re.search(r"Sex/Age\s*:\s*(Male|Female)\s*/\s*(\d+)\s*Y", full_text, re.IGNORECASE)
    if age_gender_match:
        extracted_data["Gender"] = age_gender_match.group(1)
        extracted_data["Age"] = age_gender_match.group(2)

    # 2. Hemoglobin (Test Hemoglobin ... Result 14.5)
    hb_match = re.search(r"Test\s*Hemoglobin.*?Result\s*(\d+\.?\d*)", full_text, re.DOTALL)
    if hb_match:
        extracted_data["Hemoglobin"] = hb_match.group(1)

    # 3. WBC (WBC Count ... 10570)
    wbc_match = re.search(r"WBC\s*Count\s*(\d+)", full_text)
    if wbc_match:
        extracted_data["WBC"] = wbc_match.group(1)

    # 4. Glucose (Fasting Blood Sugar ... 141.0)
    # Note: Using DOTALL to handle potential line breaks between name and result
    glucose_match = re.search(r"Fasting\s*Blood\s*Sugar.*?(\d+\.?\d*)", full_text, re.DOTALL)
    if glucose_match:
        extracted_data["Glucose"] = glucose_match.group(1)

    # 5. Cholesterol (Cholesterol ... 189.0)
    chol_match = re.search(r"Cholesterol\s+Cholesterol.*?(\d{2,3}\.?\d*)", full_text, re.DOTALL)
    if chol_match:
        extracted_data["Cholesterol"] = chol_match.group(1)

    # 6. Kidney Profile (Creatinine, Urea, BUN)
    creat_match = re.search(r"Creatinine,\s*Serum\s*(\d+\.?\d*)", full_text)
    if creat_match:
        extracted_data["Creatinine"] = creat_match.group(1)

    # Urea (looking for the specific value 18.0 format in report)
    urea_match = re.search(r"Urea\s+L?\s*(\d+\.?\d*)", full_text)
    if urea_match:
        extracted_data["Urea"] = urea_match.group(1)

    # BUN (Blood Urea Nitrogen)
    bun_match = re.search(r"Blood\s*Urea\s*Nitrogen\s+L?\s*(\d+\.?\d*)", full_text)
    if bun_match:
        extracted_data["BUN"] = bun_match.group(1)

    # 7. Electrolytes (Sodium, Potassium)
    na_match = re.search(r"Sodium\s*\(Na\+\).*?(\d{3}\.?\d*)", full_text, re.DOTALL)
    if na_match:
        extracted_data["Sodium"] = na_match.group(1)

    k_match = re.search(r"Potassium\s*\(K\+\).*?(\d\.\d+)", full_text, re.DOTALL)
    if k_match:
        extracted_data["Potassium"] = k_match.group(1)

    # 8. pH (Urine pH found in report)
    ph_match = re.search(r"PH\s+Double\s*indicator\s*(\d+\.?\d*)", full_text)
    if ph_match:
        extracted_data["pH"] = ph_match.group(1)

    return extracted_data

# --- EXECUTION ---
file_path = 'C:\AINutriCare\Data\Raw\Reports\REPORT.pdf' # Ensure this matches your file name
results = extract_medical_parameters(file_path)

print("--- Extracted Medical Parameters ---")
for key, value in results.items():
    print(f"{key}: {value}")

[INFO] Extracting text from: C:\AINutriCare\Data\Raw\Reports\REPORT.pdf

=== Extracted Parameters ===
Glucose     : 157.07 mg/dL
Insulin     : None µIU/mL
Creatinine  : 0.83 mg/dL
Urea (BUN)  : None mg/dL
Sodium      : 143.00 mmol/L
Potassium   : 4.90 mmol/L
Hemoglobin  : 14.5 g/dL
WBC         : 10570 /cmm
Lactate     : None mmol/L
pH          : 6.0 
Age         : 41 years
Gender      : Male 
Cholestrol  : 189.0 mg/dL
[INFO] Saved extracted parameters to: extracted_parameters.csv


In [ ]:
import numpy as np
import json
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Layer
import os

# ==========================================
# 1. Setup & Configuration
# ==========================================
# Path to the extracted data from the previous step
JSON_INPUT = "patient_vitals.json" 
MODEL_PATH = r"C:\AINutriCare\Notebooks\Milestone_2\LSTM\attention_lstm.h5"
SCALER_PATH = r"C:\AINutriCare\Data\Transformed\X_final.npy"

MODEL_FEATURES = [
    "Heart Rate", "MAP", "Respiratory Rate", "Temperature", 
    "Glucose", "Creatinine", "BUN", "Sodium", "Potassium", "Hemoglobin", "WBC", "Lactate",
    "Fluid Balance", "Vasopressors", "Sedatives", "Antibiotics", "Insulin"
]

# Defaults for missing vitals (Outpatient assumption)
DEFAULTS = {
    'Heart Rate': 75, 'MAP': 90, 'Respiratory Rate': 16, 'Temperature': 98.4,
    'Lactate': 1.0, 'Fluid Balance': 0, 'Vasopressors': 0, 'Sedatives': 0, 
    'Antibiotics': 0, 'Insulin': 0
}

# ==========================================
# 2. Model Resources (Same as before)
# ==========================================
class SimpleAttention(Layer):
    def __init__(self, units=64, **kwargs):
        super(SimpleAttention, self).__init__(**kwargs)
        self.units = units
    def get_config(self):
        config = super(SimpleAttention, self).get_config()
        config.update({"units": self.units})
        return config
    def build(self, input_shape):
        self.W1 = self.add_weight(name='att_w1', shape=(input_shape[-1], self.units), initializer='glorot_uniform')
        self.W2 = self.add_weight(name='att_w2', shape=(self.units, 1), initializer='glorot_uniform')
        self.b1 = self.add_weight(name='att_b1', shape=(self.units,), initializer='zeros')
        super(SimpleAttention, self).build(input_shape)
    def call(self, x):
        h = tf.nn.tanh(tf.matmul(x, self.W1) + self.b1)
        e = tf.squeeze(tf.matmul(h, self.W2), -1)
        alpha = tf.nn.softmax(e)
        context = x * tf.expand_dims(alpha, -1)
        context = tf.reduce_sum(context, axis=1)
        return context, alpha

# Load Model
try:
    if os.path.exists(MODEL_PATH):
        model = load_model(MODEL_PATH, custom_objects={'SimpleAttention': SimpleAttention})
        X_ref = np.load(SCALER_PATH)
        X_flat = X_ref.reshape(-1, X_ref.shape[2])
        means = np.mean(X_flat, axis=0)
        stds = np.std(X_flat, axis=0)
        stds[stds == 0] = 1.0
        print("✅ Model Resources Loaded.")
    else:
        # Fallback for testing if files aren't present
        print("⚠️ Model files not found. Using Mock Logic for demonstration.")
        model = None
        means = np.zeros(17)
        stds = np.ones(17)
except Exception as e:
    print(f"Error: {e}")
    exit()

# ==========================================
# 3. Data Mapper (JSON -> Model Input)
# ==========================================
def map_patient_data(json_data):
    """
    Converts the JSON report keys into the 17-feature vector for the AI.
    """
    def clean_val(key):
        val = json_data.get(key, "N/A")
        if val == "N/A" or val == "Not Found": return None
        # Clean string: Remove 'H', 'L', units, etc.
        clean = str(val).replace('H', '').replace('L', '').replace('<', '').replace('>', '').replace('%', '').strip()
        try:
            return float(clean)
        except:
            return None

    # Mapping: Model Feature Name -> Report JSON Key
    mapping = {
        "Glucose": "Fasting Blood Sugar", 
        "Creatinine": "Creatinine",
        "BUN": "BUN",
        "Sodium": "Sodium",
        "Potassium": "Potassium",
        "Hemoglobin": "Hemoglobin",
        "WBC": "WBC",
        "Platelets": "Platelets"
    }

    patient_vector = []
    report_vals = {}

    for feature in MODEL_FEATURES:
        report_key = mapping.get(feature)
        value = clean_val(report_key)
        
        # Special Logic: WBC Scaling (Model expects 10.5, Report has 10570)
        if feature == "WBC" and value and value > 1000:
            value = value / 1000.0
            
        if value is not None:
            final_val = value
            report_vals[feature] = final_val
        else:
            final_val = DEFAULTS.get(feature, 0)
            
        patient_vector.append(final_val)

    return np.array(patient_vector), report_vals

# ==========================================
# 4. Prediction & Diet Logic
# ==========================================
def predict_risk(patient_vector):
    # Simulate 24h trajectory (Static Profile)
    trajectory = np.tile(patient_vector, (24, 1)) 
    scaled = (trajectory - means) / stds
    input_tensor = scaled.reshape(1, 24, 17)
    
    if model:
        prob = model.predict(input_tensor, verbose=0)[0][0]
    else:
        # Mock logic if model file missing
        gluc = patient_vector[MODEL_FEATURES.index("Glucose")]
        prob = 0.85 if gluc > 140 else 0.20
        
    return prob

def generate_diet(prob, vals, full_json):
    plans = []
    
    # AI Score Rule
    if prob > 0.6: status = "HIGH RISK (Metabolic)"
    elif prob > 0.4: status = "MODERATE RISK"
    else: status = "STABLE"

    # Specific Lab Rules
    # 1. Diabetes
    gluc = vals.get("Glucose", 0)
    hba1c = full_json.get("HbA1c", "0").replace('H','').strip()
    try: hba1c = float(hba1c)
    except: hba1c = 0
    
    if gluc > 126 or hba1c > 6.5:
        plans.append("✅ DIABETIC DIET: Low Glycemic Index.")
        plans.append("   - Target: <45g Carbs per meal. Avoid simple sugars.")
        plans.append("   - Rationale: HbA1c is " + str(hba1c) + "% (Target <6.5%)")

    # 2. Cholesterol
    chol_str = full_json.get("Total Cholesterol", "0").replace('H','').strip()
    try: chol = float(chol_str)
    except: chol = 0
    if chol > 200:
        plans.append("✅ CARDIAC DIET: Low Saturated Fat (<7% of calories).")
    
    # 3. Urine pH / Acidic
    urine_ph = full_json.get("Urine pH", "N/A")
    try: 
        if float(urine_ph) < 6.0:
            plans.append("✅ ACIDIC URINE DETECTED: Increase plant-based alkaline foods.")
    except: pass

    return status, plans

# ==========================================
# 5. MAIN EXECUTION (Automated Loading)
# ==========================================
if __name__ == "__main__":
    print(f"\n{'='*60}")
    print(f" AUTOMATED AI CLINICAL ANALYSIS")
    print(f"{'='*60}")

    # --- STEP 1: LOAD JSON DATA ---
    if not os.path.exists(JSON_INPUT):
        print(f"❌ Error: '{JSON_INPUT}' not found.")
        print("   Please run the extraction script first to generate this file.")
        exit()
        
    with open(JSON_INPUT, 'r') as f:
        patient_data = json.load(f)
        
    print(f"Patient Name: {patient_data.get('Patient Name', 'Unknown')}")
    print(f"Data Source:  {JSON_INPUT}")
    
    # --- STEP 2: MAP TO MODEL ---
    input_vector, detected_vals = map_patient_data(patient_data)
    
    # --- STEP 3: PREDICT ---
    risk_score = predict_risk(input_vector)
    
    # --- STEP 4: GENERATE PRESCRIPTION ---
    status, prescription = generate_diet(risk_score, detected_vals, patient_data)
    
    # --- STEP 5: OUTPUT REPORT ---
    print(f"\n[1] AI RISK PREDICTION")
    print(f"    Score: {risk_score:.2%} | Status: {status}")
    
    print(f"\n[2] KEY DRIVERS (Extracted)")
    for k, v in detected_vals.items():
        # Only show values that differ from defaults
        if v != DEFAULTS.get(k, 0):
            print(f"    - {k}: {v}")
    
    print(f"\n[3] PERSONALIZED DIET PLAN")
    if not prescription:
        print("    - Standard Healthy Diet (No specific restrictions detected).")
    else:
        for item in prescription:
            print(f"    {item}")
            
    print("-" * 60)

✅ Model Resources Loaded.

 AUTOMATED AI CLINICAL ANALYSIS
Patient Name: Unknown
Data Source:  patient_vitals.json



[1] AI RISK PREDICTION
    Score: 44.03% | Status: MODERATE RISK

[2] KEY DRIVERS (Extracted)
    - Glucose: 141.0
    - Creatinine: 0.83
    - BUN: 8.41
    - Potassium: 4.9
    - Hemoglobin: 14.5

[3] PERSONALIZED DIET PLAN
    ✅ DIABETIC DIET: Low Glycemic Index.
       - Target: <45g Carbs per meal. Avoid simple sugars.
       - Rationale: HbA1c is 25.0% (Target <6.5%)
------------------------------------------------------------


In [16]:
import numpy as np
import json
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Layer
import os

# ==========================================
# 1. Configuration & Defaults
# ==========================================
JSON_INPUT = "patient_vitals.json"
MODEL_PATH = r"C:\AINutriCare\Notebooks\Milestone_2\LSTM\attention_lstm.h5"
SCALER_PATH = r"C:\AINutriCare\Data\Transformed\X_final.npy"

# The 17 Features the Model was trained on
MODEL_FEATURES = [
    "Heart Rate", "MAP", "Respiratory Rate", "Temperature", 
    "Glucose", "Creatinine", "BUN", "Sodium", "Potassium", "Hemoglobin", "WBC", "Lactate",
    "Fluid Balance", "Vasopressors", "Sedatives", "Antibiotics", "Insulin"
]

# Baseline for missing vitals (Resting/Stable assumption)
DEFAULTS = {
    'Heart Rate': 75, 'MAP': 90, 'Respiratory Rate': 16, 'Temperature': 98.4,
    'Lactate': 1.0, 'Fluid Balance': 0, 'Vasopressors': 0, 'Sedatives': 0, 
    'Antibiotics': 0, 'Insulin': 0
}

# ==========================================
# 2. Load Model Resources
# ==========================================
class SimpleAttention(Layer):
    def __init__(self, units=64, **kwargs):
        super(SimpleAttention, self).__init__(**kwargs)
        self.units = units
    def get_config(self):
        config = super(SimpleAttention, self).get_config()
        config.update({"units": self.units})
        return config
    def build(self, input_shape):
        self.W1 = self.add_weight(name='att_w1', shape=(input_shape[-1], self.units), initializer='glorot_uniform')
        self.W2 = self.add_weight(name='att_w2', shape=(self.units, 1), initializer='glorot_uniform')
        self.b1 = self.add_weight(name='att_b1', shape=(self.units,), initializer='zeros')
        super(SimpleAttention, self).build(input_shape)
    def call(self, x):
        h = tf.nn.tanh(tf.matmul(x, self.W1) + self.b1)
        e = tf.squeeze(tf.matmul(h, self.W2), -1)
        alpha = tf.nn.softmax(e)
        context = x * tf.expand_dims(alpha, -1)
        context = tf.reduce_sum(context, axis=1)
        return context, alpha

try:
    if os.path.exists(MODEL_PATH):
        model = load_model(MODEL_PATH, custom_objects={'SimpleAttention': SimpleAttention})
        # Load scaler stats (Mean/Std) to normalize the input
        X_ref = np.load(SCALER_PATH)
        X_flat = X_ref.reshape(-1, X_ref.shape[2])
        means = np.mean(X_flat, axis=0)
        stds = np.std(X_flat, axis=0)
        stds[stds == 0] = 1.0
        print("✅ AI Model & Scaler Loaded.")
    else:
        print("⚠️ Model files not found. Using Rule-Based Logic for Demo.")
        model = None
        means = np.zeros(17)
        stds = np.ones(17)
except Exception as e:
    print(f"Error loading model: {e}")
    exit()

# ==========================================
# 3. Data Mapper (JSON -> AI Vector)
# ==========================================
def map_json_to_vector(json_data):
    """
    Maps text keys (e.g., 'Fasting Blood Sugar') to Model Features (e.g., 'Glucose').
    """
    def clean(val):
        if val in ["Not Found", "N/A", None]: return None
        # Clean string: Remove 'H', 'L', '<', '>', units
        s = str(val).replace('H','').replace('L','').replace('<','').replace('>','').strip()
        try: return float(s)
        except: return None

    # Mapping Dictionary
    mapping = {
        "Glucose": "Fasting Blood Sugar",
        "Creatinine": "Creatinine",
        "BUN": "BUN",
        "Sodium": "Sodium",
        "Potassium": "Potassium",
        "Hemoglobin": "Hemoglobin",
        "WBC": "WBC"
    }

    vector = []
    extracted_vals = {}

    for feature in MODEL_FEATURES:
        report_key = mapping.get(feature)
        val = clean(json_data.get(report_key))
        
        # Scaling Fix: WBC in report is often /cmm (10570), Model expects k/uL (10.57)
        if feature == "WBC" and val and val > 1000:
            val = val / 1000.0

        if val is not None:
            vector.append(val)
            extracted_vals[feature] = val
        else:
            vector.append(DEFAULTS.get(feature, 0))

    return np.array(vector), extracted_vals

# ==========================================
# 4. Diet Prescription Engine
# ==========================================
def generate_diet_plan(prob, vals, full_data):
    plan = {
        "Status": "STABLE",
        "Diet_Type": "Standard Balanced Diet",
        "Macros": {"Carbs": "50%", "Protein": "20%", "Fat": "30%"},
        "Restrictions": [],
        "Rationale": []
    }

    # --- 1. RISK ASSESSMENT ---
    if prob > 0.60:
        plan["Status"] = "HIGH RISK (Metabolic Instability)"
        plan["Restrictions"].append("Strict Monitoring Required")
    elif prob > 0.40:
        plan["Status"] = "MODERATE RISK"

    # --- 2. DIABETES LOGIC ---
    glucose = vals.get("Glucose", 0)
    hba1c_str = full_data.get("HbA1c", "0").replace('H','').strip()
    try: hba1c = float(hba1c_str)
    except: hba1c = 0
    
    if glucose > 126 or hba1c > 6.5:
        plan["Diet_Type"] = "Diabetic Protocol (Low Glycemic Index)"
        plan["Macros"] = {"Carbs": "35-40%", "Protein": "25%", "Fat": "35-40%"}
        plan["Restrictions"].append("Limit Net Carbs < 45g per meal")
        plan["Restrictions"].append("Avoid High GI Fruits (Banana, Mango)")
        plan["Rationale"].append(f"HbA1c is {hba1c}% (Diabetic Range > 6.5%)")
        plan["Rationale"].append(f"Fasting Glucose is {glucose} mg/dL (High)")

    # --- 3. RENAL LOGIC ---
    creat = vals.get("Creatinine", 0)
    if creat > 1.3:
        plan["Restrictions"].append("Restrict Potassium (Bananas, Potatoes)")
        plan["Restrictions"].append("Limit Protein to 0.8g/kg")
        plan["Rationale"].append(f"Creatinine elevated ({creat} mg/dL)")

    # --- 4. LIPID LOGIC ---
    chol_str = full_data.get("Total Cholesterol", "0").replace('H','').strip()
    try: chol = float(chol_str)
    except: chol = 0
    
    if chol > 200:
        plan["Restrictions"].append("Low Saturated Fat (<7% total cal)")
        plan["Rationale"].append(f"Cholesterol is {chol} mg/dL")

    return plan

# ==========================================
# 5. Execution Loop
# ==========================================
if __name__ == "__main__":
    print(f"\n{'='*60}")
    print(f" FINAL CLINICAL DIET RECOMMENDATION")
    print(f"{'='*60}")

    # 1. Load Extracted Data
    if not os.path.exists(JSON_INPUT):
        print("Error: JSON input not found. Run the Extraction script first.")
        exit()
        
    with open(JSON_INPUT, 'r') as f:
        patient_data = json.load(f)

    print(f"Patient: {patient_data.get('Patient Name', 'Unknown')}")
    print(f"Age/Sex: {patient_data.get('Age', '?')} / {patient_data.get('Gender', '?')}")

    # 2. AI Prediction
    input_vec, active_vals = map_json_to_vector(patient_data)
    
    # Simulate 24h trajectory for the LSTM
    trajectory = np.tile(input_vec, (24, 1))
    scaled_input = (trajectory - means) / stds
    scaled_input = scaled_input.reshape(1, 24, 17)
    
    if model:
        risk_score = model.predict(scaled_input, verbose=0)[0][0]
    else:
        # Fallback Logic if model missing
        gluc = active_vals.get("Glucose", 100)
        risk_score = 0.75 if gluc > 140 else 0.25

    # 3. Generate Diet
    diet = generate_diet_plan(risk_score, active_vals, patient_data)

    # 4. Output Report
    print(f"\n[1] AI RISK ANALYSIS")
    print(f"    Mortality Risk Score: {risk_score:.2%}")
    print(f"    Clinical Category:    {diet['Status']}")
    
    print(f"\n[2] NUTRITION PRESCRIPTION")
    print(f"    Protocol: {diet['Diet_Type']}")
    print(f"    Target Macros: Carbs {diet['Macros']['Carbs']} | Protein {diet['Macros']['Protein']} | Fat {diet['Macros']['Fat']}")
    
    if diet['Restrictions']:
        print("\n[3] SPECIFIC RESTRICTIONS")
        for r in diet['Restrictions']:
            print(f"    ⚠️ {r}")
            
    if diet['Rationale']:
        print("\n[4] CLINICAL RATIONALE")
        for r in diet['Rationale']:
            print(f"    ℹ️ {r}")
            
    print("-" * 60)

✅ AI Model & Scaler Loaded.

 FINAL CLINICAL DIET RECOMMENDATION
Patient: Unknown
Age/Sex: 41 / Male

[1] AI RISK ANALYSIS
    Mortality Risk Score: 44.03%
    Clinical Category:    MODERATE RISK

[2] NUTRITION PRESCRIPTION
    Protocol: Diabetic Protocol (Low Glycemic Index)
    Target Macros: Carbs 35-40% | Protein 25% | Fat 35-40%

[3] SPECIFIC RESTRICTIONS
    ⚠️ Limit Net Carbs < 45g per meal
    ⚠️ Avoid High GI Fruits (Banana, Mango)

[4] CLINICAL RATIONALE
    ℹ️ HbA1c is 25.0% (Diabetic Range > 6.5%)
    ℹ️ Fasting Glucose is 141.0 mg/dL (High)
------------------------------------------------------------
